# Steady State Over Voltage Hosting Capacity for a New Generator

In this example, we calculate the hosting capacity for a new generator at bus "bus_1101" using the steady state over voltage (overvoltage) metric. This example uses Python to control OpenDSS.

1. OpenDSS: <https://www.epri.com/pages/sa/opendss>
2. py-dss-interface: <https://pypi.org/project/py-dss-interface/>

Importing third-party Python packages and modules from this repository.

In [58]:
# -*- coding: utf-8 -*-
# @Author  : Paulo Radatz
# @Email   : pradatz@epri.com

In [59]:
import py_dss_interface
import numpy as np

In [60]:
from FeederCondition import FeederCondition
from HCSteps import HCSteps
from HCSeetings import HCSettings

## Feeder Model

Hosting capacity analysis requires a feeder model. You can find the feeder used in this example at <https://github.com/rkerestes/IEEE1729DynamicsTF/tree/main/HCA/HCABase.dss>

In [61]:
dss_file = r"C:\PauloRadatz\GitHub\IEEE1729DynamicsTF\HCA\HCABase.dss"

In [62]:
dss = py_dss_interface.DSS()
dss.text(f"compile [{dss_file}]")
dss.text("edit EnergyMeter.m2 enabled=no")

''

## Bus Selection

The bus selection is indicated below.

In [63]:
bus = "bus_1101"

## Load and Connected Generation Models

In this example, generator and load models are considered constant power. You can change this in OpenDSS if desired.

In [64]:
FeederCondition.set_load_model(dss)
FeederCondition.set_generator_model(dss)

## Feeder Condition | Hosting Capacity Impact Factors

The overvoltage capacity should evaluate the voltage at each bus under worst-case conditions. In this particular case, the worst-case condition is maximum generation and minimum load. The maximum generation includes the maximum output of the new generator plus any connected generators. On the other hand, the minimum load is defined as the lowest load consumption across the entire timeframe. If more information on the coincidence of load consumption, connected generators' generation, and the new generator's generation is available, it could be taken into account to set the maximum generation and minimum load accordingly.

In this case, the connected generators are assumed to generate at 100% capacity, and the minimum load is considered to be 20% of the total load.

Voltage regulation equipment, such as on-load tap changers (OLTC), line regulators, and capacitors, plays a crucial role in overvoltage hosting capacity.

For OLTCs and line regulators, we need to consider two aspects: first, how we want to account for the equipment's operation, and second, how we want to consider its initial condition.

Regarding the operation of the equipment, we can assume it either operates or doesn't. If the new generation's active power output can change faster than the equipment's operation, we can assume they don't operate during the hosting capacity assessment. Conversely, if the new generation's active power output isn't faster than the equipment's operation, we can allow it to operate during the assessment.

For the initial condition of the equipment, it's important to understand that the actual voltage of the controlled bus, under normal conditions, is within the bandwidth—a range around the voltage setpoint. The initial condition can be considered in various ways: it can be the voltage setpoint, the top or bottom of the bandwidth, or another point in between, perhaps determined by using the base case power flow results.

In this example, we'll consider that the initial point is the top of the bandwidth, the worse-case condition, and the regulator is slower than the change in active power of the new generator. Therefore, it won't be allowed to operate during the hosting capacity analysis.

For capacitors, we can also consider whether they are ON or OFF. We can assume the capacitor is OFF, as the minimum demand condition typically results in switched capacitors being OFF.

In [65]:
hc_settings = HCSettings(load_mult=0.2, capacitor=False, add_existing_ger=True, mult_existing_gen=1, push_reg=1)

In [66]:
FeederCondition.set_capacitor_status(dss, capacitor=hc_settings.capacitor)
FeederCondition.set_load_level_condition(dss, load_mult=hc_settings.load_mult)
FeederCondition.push_regulators(dss, push_reg=hc_settings.push_reg)
FeederCondition.consider_existing_gen(dss, hc_settings.add_existing_ger)

## Adding a New Generator to the Feeder Model

We add a generator model as constant power to the feeder model. In this example, the generator is 3-phase.

In [67]:
dss.circuit.set_active_bus(bus)
kv = dss.bus.kv_base * np.sqrt(3)
gen_bus = {"gen": dss.bus.name}
gen_kv = {"gen": kv}
HCSteps.add_gen(dss, gen_bus, gen_kv)

## Hosting Capacity Iterative Process

The hosting capacity iterative process incrementally increases the new generator’s size and performs a power flow check for violations. The process ends if there's a voltage violation (a bus with voltage exceeding the overvoltage threshold, in this example 1.05 pu) or if the generator size reaches the maximum analyzed size (in this case, 10 MW).

Thus, the hosting capacity corresponds to the generator size from the previous iteration before the violation.

In [68]:
i = 0
while i * HCSteps.step_kw < HCSteps.max_kw:
    i = i + 1
    i_kw = i * HCSteps.step_kw
    gen_kw = {"gen": i_kw}

    # Set Penetration Level
    HCSteps.increase_gen(dss, gen_kw)

    # Perform Power flow
    HCSteps.solve_powerflow(dss)

    # Violation?
    if HCSteps.check_overvoltage_violation(dss):
        hosting_capacity_value_kw = (i - 1) * HCSteps.step_kw
        break

In [69]:
print(f"Hosting Capacity = {hosting_capacity_value_kw / 1000} MW")

Hosting Capacity = 0.0 MW


## Hosting Capacity Result

This example demonstrates how to calculate overvoltage hosting capacity using OpenDSS controlled by Python. Note that the hosting capacity result is strongly related to the feeder conditions considered for the analysis. In this case, the conditions are a load level of 20%, all generators at 100% output, line regulator locked and with top of the bandwidth initial condition, and capacitors OFF. This scenario is likely the worst condition, and the feeder might not even operate under it. Therefore, to better represent the worst condition the feeder can actually operate under, more operational information is needed. This includes control elements, load consumption, and generator production coincidence, which can then be considered in the hosting capacity analysis. However, sometimes this information is not available or known.

More impact factors can be considered in the analysis, such as smart inverter functionalities.

The model itself can also impact the hosting capacity results. For instance, whether the lines are modeled as positive and negative sequences, which load and generation models are considered, etc.